In [183]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [185]:
df = pd.read_csv('titanic.csv')

In [187]:
df = df.drop(columns = ['Name','Ticket', 'PassengerId'])

In [189]:
df.columns = df.columns.str.lower()

In [191]:
df

,survived,pclass,sex,age,sibsp,parch,fare,cabin,embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,NaN,S
887,1,1,female,19.0,0,0,30.0000,B42,S
888,0,3,female,NaN,1,2,23.4500,NaN,S
889,1,1,male,26.0,0,0,30.0000,C148,C


In [195]:
df = df.dropna(subset='embarked')

In [197]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='survived'), df.survived, test_size = 0.2, random_state = 100, stratify =df.survived)

In [193]:
X_test.isna().sum()

pclass          0
age             0
sibsp           0
parch           0
fare          178
sex_male        0
cabin_B         0
cabin_C         0
cabin_D         0
cabin_E         0
cabin_F         0
cabin_G         0
cabin_O         0
cabin_T         0
embarked_Q      0
embarked_S      0
dtype: int64

In [199]:
mean_imputer = SimpleImputer(strategy='mean')

In [201]:
mean_imputer.fit(X_train.loc[:,['age']])
X_train.loc[:,['age']] = mean_imputer.transform(X_train.loc[:,['age']])
X_test.loc[:,['age']]= mean_imputer.transform(X_test.loc[:,['age']])


X_train.loc[:,['cabin']]=X_train.loc[:,['cabin']].fillna('Other')
X_test.loc[:,['cabin']]=X_test.loc[:,['cabin']].fillna('Other')

#X_train=X_train.dropna(subset='embarked')
#X_test=X_test.dropna(subset='embarked')

In [203]:
X_test.isna().sum()

pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
cabin       0
embarked    0
dtype: int64

In [205]:
X_train['fare'] = X_train.fare.clip(X_train.fare.min(), X_train.fare.quantile(0.995))
#X_test['fare'] = X_train.fare.clip(X_train.fare.min(), X_train.fare.quantile(0.995))

In [207]:
X_train['cabin'] = X_train.cabin.str.slice(0,1)
X_test['cabin'] = X_test.cabin.str.slice(0,1)


In [209]:
pd.concat([X_train, y_train], axis=1).groupby('cabin')['survived'].mean().round(3)

cabin
A    0.417
B    0.744
C    0.612
D    0.792
E    0.783
F    0.600
G    1.000
O    0.297
T    0.000
Name: survived, dtype: float64

In [211]:
X_train.fare.quantile([0.95, 0.99, 0.995, 0.999])

0.950    112.079150
0.990    245.521220
0.995    262.501563
0.999    262.656250
Name: fare, dtype: float64

In [213]:
X_train = pd.get_dummies(X_train, drop_first = True)
X_test = pd.get_dummies(X_test, drop_first = True)

In [215]:
from sklearn.linear_model import LogisticRegression

In [217]:
log_reg = LogisticRegression(max_iter = 500)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=500)

In [219]:
X_test['cabin_T'] = False

In [221]:
X_train.columns

Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_male', 'cabin_B',
       'cabin_C', 'cabin_D', 'cabin_E', 'cabin_F', 'cabin_G', 'cabin_O',
       'cabin_T', 'embarked_Q', 'embarked_S'],
      dtype='object')

In [223]:
X_test = X_test.loc[:, ['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_male', 'cabin_B',
       'cabin_C', 'cabin_D', 'cabin_E', 'cabin_F', 'cabin_G', 'cabin_O',
       'cabin_T', 'embarked_Q', 'embarked_S']]

In [225]:
X_test.isna().sum()

pclass        0
age           0
sibsp         0
parch         0
fare          0
sex_male      0
cabin_B       0
cabin_C       0
cabin_D       0
cabin_E       0
cabin_F       0
cabin_G       0
cabin_O       0
cabin_T       0
embarked_Q    0
embarked_S    0
dtype: int64

In [227]:
pred = log_reg.predict(X_test)

In [229]:
from sklearn.metrics import accuracy_score, f1_score

In [231]:
accuracy_score(pred, y_test)

0.7696629213483146

In [233]:
f1_score(pred, y_test)

0.7050359712230215